In [1]:
import os, sys

sys.path.append("../")

from typing import List, Dict, Union, Optional, Any, Tuple, Literal

import pandas as pd
import networkx as nx
import sqlalchemy as sa

import torch

import torch_geometric
from torch_geometric.typing import EdgeType, NodeType

import torch_frame

from relbench.datasets import get_dataset_names, get_dataset
from relbench.tasks import get_task_names, get_task
from relbench.base import Database

import ctu_relational

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
%reload_ext autoreload
%autoreload 2

/home/jakub/miniconda3/envs/relational-py/lib/python3.12/site-packages/torch/__config__.py:10: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647378361/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._show_config()
/home/jakub/miniconda3/envs/relational-py/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ctu_datasets = list(filter(lambda x: x.startswith("ctu"), get_dataset_names()))

In [4]:
def get_db_schema_graph(db: Database):
    G = nx.MultiGraph()
    for tname, table in db.table_dict.items():
        G.add_node(tname)
        for fk, fktname in table.fkey_col_to_pkey_table.items():
            G.add_edge(tname, fktname, name=fk)
    return G

In [5]:
def get_info(dataset: ctu_relational.datasets.CTUDataset):
    url = dataset.get_url(
        "mariadb",
        "pymysql",
        "guest",
        "ctu-relational",
        "relational.fel.cvut.cz",
        3306,
        "meta",
    )
    with dataset.create_remote_connection(url) as conn:

        metadata = sa.MetaData()
        metadata.reflect(bind=conn.engine)

        information = metadata.tables.get("information")

        q = sa.select(
            information.c.domain,
            information.c.is_artificial,
            information.c.database_size,
        ).where(information.c.database_name == dataset.database)

        info = conn.execute(q).fetchone()

        return info._asdict()

In [6]:
info = {}
# info = pd.read_csv("ctu_dataset_info.csv").T.to_dict()

for dataset_name in ctu_datasets:

    dataset: ctu_relational.datasets.CTUDataset = get_dataset(dataset_name)

    meta = get_info(dataset)
    meta["db_size_MB"] = meta["database_size"]
    meta.pop("database_size")

    db = dataset.get_db(upto_test_timestamp=False)

    g = get_db_schema_graph(db)

    info[dataset_name] = {}

    info[dataset_name]["dataset"] = dataset_name
    info[dataset_name] = {**info[dataset_name], **meta}

    info[dataset_name]["n_tables"] = len(db.table_dict)
    info[dataset_name]["n_fks"] = len(
        [fk for t in db.table_dict.values() for fk in t.fkey_col_to_pkey_table.keys()]
    )
    info[dataset_name]["n_factual_cols"] = (
        sum([len(t.df.columns) for t in db.table_dict.values()])
        - info[dataset_name]["n_fks"]
        - info[dataset_name]["n_tables"]
    )
    info[dataset_name]["total_n_tuples"] = sum(len(t.df) for t in db.table_dict.values())
    info[dataset_name]["total_n_fk_edges"] = sum(
        t.df[fk].notna().sum()
        for t in db.table_dict.values()
        for fk in t.fkey_col_to_pkey_table.keys()
    )
    info[dataset_name]["total_ratio_edges_tuples"] = (
        info[dataset_name]["total_n_fk_edges"] / info[dataset_name]["total_n_tuples"]
    )

    info[dataset_name]["schema_diameter"] = nx.diameter(g)

    df = pd.DataFrame(info).T
    df.to_csv("./ctu-dataset-info.csv", index=False)


df = pd.DataFrame(info).T

print(df)

df.to_csv("./ctu-dataset-info.csv", index=False)

Loading Database object from /home/jakub/.cache/relbench/ctu-accidents/db...
Done in 0.60 seconds.
Loading Database object from /home/jakub/.cache/relbench/ctu-adventureworks/db...
Done in 0.38 seconds.
Loading Database object from /home/jakub/.cache/relbench/ctu-airline/db...
Done in 0.45 seconds.
Loading Database object from /home/jakub/.cache/relbench/ctu-atherosclerosis/db...
Done in 0.06 seconds.
Loading Database object from /home/jakub/.cache/relbench/ctu-basketballmen/db...
Done in 0.19 seconds.
Loading Database object from /home/jakub/.cache/relbench/ctu-basketballwomen/db...
Done in 0.04 seconds.
Loading Database object from /home/jakub/.cache/relbench/ctu-biodegradability/db...
Done in 0.02 seconds.
Loading Database object from /home/jakub/.cache/relbench/ctu-bupa/db...
Done in 0.03 seconds.
Loading Database object from /home/jakub/.cache/relbench/ctu-carcinogenesis/db...
Done in 0.02 seconds.
Loading Database object from /home/jakub/.cache/relbench/ctu-cde/db...
Done in 0.07